# Comparison EDSR, WDSR and SRGAN

## Описание задачи

Рассмотреть модели EDSR, WDSR и SRGAN для Super Resolution из репозитория https://github.com/krasserm/super-resolution

Ссылка на обученный датасет: https://data.vision.ee.ethz.ch/cvl/DIV2K/

### Import

Mount к локальному Google Disk

In [10]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [11]:
import sys
import os
sys.path.append(os.path.abspath('/content/drive/My Drive/Colab Notebooks/super-resolution'))

In [16]:
!pip install tensorflow_addons

Could not fetch URL https://pypi.org/simple/tensorflow-addons/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/tensorflow-addons/ (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),)) - skipping
ERROR: Could not find a version that satisfies the requirement tensorflow_addons (from versions: none)
ERROR: No matching distribution found for tensorflow_addons


In [ ]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
# tf.enable_eager_execution()

import cv2

%matplotlib inline

In [ ]:
# Location of model weights (needed for demo)
weights_dir = '/content/drive/My Drive/Colab Notebooks/super-resolution/weights/'
weights_file = lambda filename: os.path.join(weights_dir, filename)

os.makedirs(weights_dir, exist_ok=True)

### utils

Здесь собраны следующие вспомогательные функции:
 - save_picture - подгрузка изображения
 - resolve_and_plot - применить SISR(single image super resolution) к картинке
 - result_plot - визуальное сравнение результатов работы всех моделей друг с другом и оригиналом

In [ ]:
from model import resolve_single
from utils import load_image, plot_sample

# name of dir for save results
output_dir = '/content/drive/My Drive/Colab Notebooks/super-resolution/output/'
# name of dir for take pictures
example_dir = '/content/drive/My Drive/Colab Notebooks/super-resolution/demo'

def save_picture(lr_image_path, filename, model):
    lr = load_image(lr_image_path)
    sr = resolve_single(model, lr)

    cv2.imwrite(os.path.join(output_dir, filename), sr.numpy())

def resolve_and_plot(lr_image_path, model, name):
    lr = load_image(lr_image_path)
    sr = resolve_single(model, lr)

    plt.figure(figsize=(20, 20))
    
    images = [lr, sr]
    titles = ['LR', name]
    positions = [1, 2]
    
    for i, (img, title, pos) in enumerate(zip(images, titles, positions)):
        plt.subplot(2, 2, pos)
        plt.imshow(img)
        plt.title(title)
        plt.xticks([])
        plt.yticks([])

def result_plot(images):
    plt.figure(figsize=(20, 20))

    titles = ['LR', 'EDSR', 'WDSR', 'SRGAN']
    positions = [1, 2, 3, 4]
    
    for i, (img, title, pos) in enumerate(zip(images, titles, positions)):
        plt.subplot(2, 2, pos)
        plt.imshow(img)
        plt.title(title)
        plt.xticks([])
        plt.yticks([])

## Run models

### EDSR

In [ ]:
# Number of residual blocks
depth = 16

# Super-resolution factor
scale = 4

# Downgrade operator
downgrade = 'bicubic'

In [ ]:
from model.edsr import edsr

model_edsr = edsr(scale=scale, num_res_blocks=depth)
model_edsr.load_weights(weights_file('EDSR/weights.h5'))

In [ ]:
resolve_and_plot(os.path.join(example_dir, '0829x4-crop.png'), model_edsr, 'EDSR')
save_picture(os.path.join(example_dir, 'rt.jpg'), 'picture_edsr.jpg', model_edsr)

### WDSR

In [ ]:
# Number of residual blocks
depth = 32

# Super-resolution factor
scale = 4

# Downgrade operator
downgrade = 'bicubic'

In [ ]:
from model.wdsr import wdsr_b

model_wdsr = wdsr_b(scale=scale, num_res_blocks=depth)
model_wdsr.load_weights(weights_file('WDSR/weights.h5'))

In [ ]:
resolve_and_plot(os.path.join(example_dir, '0829x4-crop.png'), model_wdsr, 'WDSR')
save_picture(os.path.join(example_dir, 'rt.jpg'), 'picture_wdsr.jpg', model_wdsr)

### SRGAN

In [ ]:
from model.srgan import generator, discriminator

pre_generator = generator()
gan_generator = generator()

pre_generator.load_weights(weights_file('SRGAN/pre_generator.h5'))
gan_generator.load_weights(weights_file('SRGAN/gan_generator.h5'))

In [ ]:
resolve_and_plot(os.path.join(example_dir, '0829x4-crop.png'), gan_generator, 'SRGAN')
save_picture(os.path.join(example_dir, 'rt.jpg'), 'picture_srgan.jpg', gan_generator)

## Results

In [ ]:
pic_0 = cv2.imread(os.path.join(example_dir, 'rt.jpg'))
pic_1 = cv2.imread(os.path.join(output_dir, 'picture_edsr.jpg'))
pic_2 = cv2.imread(os.path.join(output_dir, 'picture_wdsr.jpg'))
pic_3 = cv2.imread(os.path.join(output_dir, 'picture_srgan.jpg'))

images = [pic_0, pic_1, pic_2, pic_3]
result_plot(images)

### Сравнение по метрикам

## Conclusion